# API Foursquares

## Arrondissements parisiens avec le prix de l'immobilier

In [1]:
import pandas as pd
import requests
import geocoder
import numpy as np 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

### 1. Le prix de l'immobilier à Paris (Average price / m2)

In [2]:
paris_arr = pd.read_csv('C:/Users/Seohyun/Desktop/paris.csv')

In [3]:
paris_arr.head()

,PostalCode,Price/m2,Latitude,Longitude
0,75001,13940,48.859,2.342
1,75002,12250,48.865,2.343
2,75003,12650,48.864,2.361
3,75004,13660,48.860,2.351
4,75005,12750,48.845,2.347


### 2. FourSquare Venue Data à Paris

In [ ]:
CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20200615' # Foursquare API version

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'
        .format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
paris_venues = getNearbyVenues(names=paris_arr['PostalCode'],
                                   latitudes=paris_arr['Latitude'],
                                   longitudes=paris_arr['Longitude'],
                                   )


In [12]:
paris_venues.to_csv('paris_commerce.csv')
paris_venues.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,75001,48.859,2.342,Place du Louvre,48.859841,2.340822,Plaza
1,75001,48.859,2.342,Église Saint-Germain-l'Auxerrois (Église Saint...,48.859520,2.341306,Church
2,75001,48.859,2.342,Coffee Crêpes,48.858841,2.340802,Coffee Shop
3,75001,48.859,2.342,Le Fumoir,48.860424,2.340868,Cocktail Bar
4,75001,48.859,2.342,Place Dauphine,48.856640,2.342362,Plaza


In [22]:
paris_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
75001,56,56,56,56,56,56
75002,100,100,100,100,100,100
75003,92,92,92,92,92,92
75004,67,67,67,67,67,67
75005,100,100,100,100,100,100
75006,55,55,55,55,55,55
75007,78,78,78,78,78,78
75008,45,45,45,45,45,45
75009,52,52,52,52,52,52


### 3. Analyser chaque arrondissement à Paris

In [23]:
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['District'] = paris_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,District,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,75001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,75001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,75001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,75001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,75001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
paris_grouped = paris_onehot.groupby('District').mean().reset_index()
paris_grouped
paris_grouped.to_json('paris_grouped',orient='table')

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['District']
for idx in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(idx+1, indicators[idx]))
    except:
        columns.append('{}th Most Common Venue'.format(idx+1))

district_venues_sorted = pd.DataFrame(columns=columns)
district_venues_sorted['District'] = paris_grouped['District']

for idx in np.arange(paris_grouped.shape[0]):
    district_venues_sorted.iloc[idx, 1:] = return_most_common_venues(paris_grouped.iloc[idx, :], num_top_venues)

In [32]:

district_venues_sortedto_json('paris_venues',orient='table')

### 4. Clustering des arrondissments à Paris

In [91]:
kclusters = 2

paris_grouped_clustering = paris_grouped.drop('District', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0])

In [33]:

paris_arr.rename(columns = {'PostalCode' : 'District'}, inplace = True)
paris_merged = paris_arr
paris_merged = paris_arr.join(district_venues_sorted.set_index('District'), on='District')


paris_merged
paris_merged.to_json('paris_merged',orient='table')

In [100]:
map_clusters = folium.Map(location=[48.8566969, 2.3514616], zoom_start=12)

rainbow = ['blue', 'red']

markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['District'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Examin clusters

In [104]:
cluster1 = paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[0] + list(range(5, paris_merged.shape[1]))]]
cluster1

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75001,Café,Hotel,French Restaurant,Clothing Store,Plaza,Tea Room,Wine Bar,Historic Site,Italian Restaurant,Bakery
1,75002,French Restaurant,Wine Bar,Bakery,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Cheese Shop,Sandwich Place,Optical Shop,Bistro
2,75003,Wine Bar,Japanese Restaurant,Art Gallery,French Restaurant,Coffee Shop,Seafood Restaurant,Italian Restaurant,Bakery,Sandwich Place,Cocktail Bar
3,75004,Bakery,Hotel,Art Gallery,French Restaurant,Plaza,Park,Historic Site,Coffee Shop,Ice Cream Shop,Restaurant
4,75005,French Restaurant,Hotel,Café,Pub,Italian Restaurant,Bar,Indie Movie Theater,Bakery,Wine Bar,Creperie
7,75008,Hotel,French Restaurant,Italian Restaurant,Café,Thai Restaurant,Bar,Bakery,Wine Shop,Theater,Corsican Restaurant
8,75009,Hotel,French Restaurant,Pedestrian Plaza,Coffee Shop,Creperie,Cocktail Bar,Italian Restaurant,Indie Movie Theater,Gourmet Shop,Gym / Fitness Center
9,75010,French Restaurant,Cocktail Bar,Breakfast Spot,Hotel,Seafood Restaurant,Salad Place,Art Gallery,Bakery,African Restaurant,Korean Restaurant
10,75011,French Restaurant,Bar,Bistro,Cocktail Bar,Japanese Restaurant,Café,Pastry Shop,Italian Restaurant,Bakery,Coffee Shop
11,75012,Hotel,Bistro,French Restaurant,Supermarket,Chinese Restaurant,Café,Garden,Sporting Goods Shop,Furniture / Home Store,Sandwich Place


In [105]:
cluster2 = paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[0] + list(range(5, paris_merged.shape[1]))]]
cluster2

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,75006,French Restaurant,Hotel,Bakery,Fountain,Wine Bar,Chocolate Shop,Garden,Ice Cream Shop,Restaurant,Art Gallery
6,75007,French Restaurant,Hotel,Café,Bakery,Coffee Shop,Bistro,Italian Restaurant,Art Museum,Japanese Restaurant,Tea Room
13,75014,French Restaurant,Hotel,Bar,Italian Restaurant,Bakery,Bistro,Restaurant,Outdoor Sculpture,Garden,Southwestern French Restaurant
15,75016,French Restaurant,Bakery,Café,Italian Restaurant,Hotel,Hotel Bar,Pizza Place,Chinese Restaurant,Restaurant,Sandwich Place
18,75019,French Restaurant,Pool,Bus Stop,Italian Restaurant,Bar,Bakery,Supermarket,Restaurant,Bike Rental / Bike Share,Spa


### Conclusion

On voit que le cluster 1 a des quartiers plus commerciaux, touristiques ou de loisirs tandis que le cluster 2 a des quartiers résidentiels. Concernant le prix de l'appartement dans chaque arrondissement de Paris, par exemple, si le prix du 6e arrondissement est trop élevé on peut trouver une alternative comme le 14e ou le 19e arrondissement qui sont moins chères, mais qui ont un état similaire avec le 6e arrondissement.